In [5]:
import torch
import torchvision
import torch.nn.functional as F
import os
import numpy as np
import cv2
import sys
import os.path
import torchvision.transforms as transforms
import torch.utils.data as data
from CONFIG import *

In [6]:
class yolo_data_preprocessing(data.Dataset):
    def __init__(self,img_dir,ann_dir,train,transform):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.train = train
        self.transform = transform 
        
        with open(ann_dir) as ann_file:
            content = f.readlines()
            
        self.image_names = []
        tracking_num_bboxes = 0
        self.boundingboxes = []
        self.class_labels = []
        self.num_images = 0
        
        for line in content:
            bbox_ = []
            class_lbl = []            
            self.num_images = self.num_images + 1
            line = line.strip().split()
            self.image_names += line[0]
            tracking_num_bboxes = line[1]
            nums = arange(tracking_num_bboxes)
            bbox_content = line[3:]    
            
            for i in nums:
                bbox_.append([float(bbox_content[i*num_params]),float(bbox_content[i*num_params+1]),float(bbox_content[i*num_params+2]),float(bbox_content[i*num_params+3])])
                class_lbl.append(int(bbox_content[i*num_params+4]))
                
            bbox_ = Torch.Tensor(bbox_)
            class_lbl = Torch.LongTensor(class_lbl)
            self.boundingboxes.append(bbox_)
            self.class_labels.append(class_lbl)
        
    def __getitem__(self,img_index):
        image = cv2.imread(os.path.join(self.img_dir+self.image_names[img_index]))
        width,height,depth = image.shape
        img_boxes = self.boundingboxes[img_index] / torch.Tensor([width,height,width,height]).expand_as(self.boundingboxes[0])
        img_labels = self.class_labels[img_index]
        image = process_image(image)
        ground_truth = torch.zeros([cells_per_row,cells_per_row])
        width = img_boxes[:,2] - img_boxes[:,0]
        height = img_boxes[:,3] - img_boxes[:,1]
        width_height = img_boxes[:,2:] - img_boxes[:,:2]
        img_center_x = (img_boxes[:,0] + img_boxes[:,2])/2
        img_center_y = (img_boxes[:,1] + img_boxes[:,3])/2
        for i in range(img_center_x.size()):
            first = centers[i]*cells_per_row
            coords = first.ceil() - 1
            coord_1 = int(coords[1])
            coord_2 = int(coords[0])
            ground_truth[coord_1,coord_2,conf1_index] = 1
            ground_truth[coord_1,coord_2,conf2_index] = 1
            ground_truth[coord_1,coord_2,int(self.class_labels[i])+conf2_index] = 1
            scaled_coords = coords/7.0 
            scaled = (first - scaled_coords)*cells_per_row
            ground_truth[coord_1,coord_2,2:4] = width_height[i]
            ground_truth[coord_1,coord_2,:2] = scaled
            ground_truth[coord_1,coord_2,7:9] = width_height[i]
            ground_truth[coord_1,coord_2,5:7] = scaled
        self.target_predictions = ground_truth
            
    def process_image(image):
        image = self.BGR2RGB(image)
        image = self.subMean(image,self.mean)
        image = cv2.resize(image,(size_image,size_image))
        return image